# Enums

## C-Style Enums

In [2]:
#[derive(Debug)]
enum Compass {
    North,
    East,
    South,
    West
}

let direction = Compass::North;

println!("direction = {:?}", direction);

direction = North


* In memory, values of C-style enums are stored as integers. 

In [3]:
use std::mem::size_of;

assert!(size_of::<Compass>() == 1);

### Importing constructors

In [4]:
use self::Compass::*; // self - when enum type is defined in current module; otherwise name of a module

let direction = North;

### Using Integral Values

In [5]:
#[derive(Debug)]
enum HttpStatus {
    Ok = 200,
    NotModified = 304,
    NotFound = 404
}

assert!(size_of::<HttpStatus>() == 2);

### Casting to Integer

In [6]:
let ok_code = HttpStatus::Ok as i32;
assert!(ok_code == 200);

assert!(HttpStatus::NotFound as i32 == 404);

### Casting from Integer

In [7]:
fn http_status_from_u32(status: u32) -> Option<HttpStatus> {
    match status {
        200 => Some(HttpStatus::Ok),
        304 => Some(HttpStatus::NotModified),
        404 => Some(HttpStatus::NotFound),
        _ => None
    }
}

In [8]:
http_status_from_u32(404)

Some(NotFound)

### Deriving From Common Traits

* As with structs, the compiler will implement features like the `==` operator for you, but you have to ask.

In [9]:
#[derive(Copy, Clone, Debug, PartialEq)]
enum TimeUnit {
    Seconds, Minutes, Hours, Days, Months, Years
}

In [10]:
assert!(TimeUnit::Seconds == TimeUnit::Seconds);

### Implementing Methods for Enums

In [11]:
impl TimeUnit {
    /// Return the plural noun for this time unit.
    fn plural(self) -> &'static str {
        match self {
            TimeUnit::Seconds => "seconds",
            TimeUnit::Minutes => "minutes",
            TimeUnit::Hours => "hours",
            TimeUnit::Days => "days",
            TimeUnit::Months => "months",
            TimeUnit::Years => "years"
        }
    }

    /// Return the singular noun for this time unit.
    fn singular(self) -> &'static str {
        self.plural().trim_right_matches('s')
    }
}

In [12]:
let time_unit = TimeUnit::Days;

time_unit.plural()

"days"

## Enums With Data

In [13]:
#[derive(Clone, Debug, PartialEq)]
enum Message {
    Quit, // no data associated
    Move{ x: i32, y: i32 }, // named fields - like a struct
    Write(String), // single String
    ChangeColor(i32, i32, i32) // tuple struct of three values
}

In [14]:
let mut msg = Message::Quit;
println!("msg = {:?}", msg);

msg = Message::Move{ x: 100, y: 200};
println!("msg = {:?}", msg);

msg = Message::Write(String::from("TEXT"));
println!("msg = {:?}", msg);

msg = Message::ChangeColor(255, 0, 255);
println!("msg = {:?}", msg);

msg = Quit
msg = Move { x: 100, y: 200 }
msg = Write("TEXT")
msg = ChangeColor(255, 0, 255)


In [15]:
impl Message {
    fn call(&self) {
        match *self {
            Message::Quit => println!("Quit"),
            Message::Move{ x, y } => println!("Move to x={}, y={}", x, y),
            Message::Write(ref text) => println!("Write '{}'", text),
            Message::ChangeColor(r, g, b) => println!("Change color to r={}, g={}, b={}", r, g, b)
        }
    }
}

In [16]:
msg.call();

Change color to r=255, g=0, b=255


## Generic Enums

### Option

In [ ]:
enum Option<T> {
    None,
    Some(T)
}

### Result

In [ ]:
enum Result<T, E> { 
    Ok(T), 
    Err(E) 
} 

### Real-World Example - BinaryTree

In [17]:
// An ordered collection of `T`s.
enum BinaryTree<T> {
    Empty,
    NonEmpty(Box<TreeNode<T>>)
}

// A part of a BinaryTree.
struct TreeNode<T> {
    element: T,
    left: BinaryTree<T>,
    right: BinaryTree<T>
}

In [18]:
use self::BinaryTree::*;

let jupyter_tree = NonEmpty(Box::new(TreeNode {
    element: "Jupyter",
    left: Empty,
    right: Empty
}));

let mercury_tree = NonEmpty(Box::new(TreeNode {
    element: "Jupyter",
    left: Empty,
    right: Empty
}));

let venus_tree = NonEmpty(Box::new(TreeNode {
    element: "Jupyter",
    left: Empty,
    right: Empty
}));

let mars_tree = NonEmpty(Box::new(TreeNode {
    element: "Saturn",
    left: jupyter_tree,
    right: mercury_tree
}));

let uranus_tree = NonEmpty(Box::new(TreeNode {
    element: "Mars",
    left: Empty,
    right: venus_tree
}));

// the root
let saturn_tree = NonEmpty(Box::new(TreeNode {
    element: "Saturn",
    left: mars_tree,
    right: uranus_tree
}));

## Patterns

In [19]:
enum RoughTime {
    InThePast(TimeUnit, u32),
    JustNow,
    InTheFuture(TimeUnit, u32)
}

In [20]:
fn rough_time_to_english(rt: RoughTime) -> String {
    match rt {
        RoughTime::InThePast(units, 1) => format!("{} ago", units.singular()),
        RoughTime::InThePast(units, count) => format!("{} {} ago", count, units.plural()),
        RoughTime::JustNow => format!("just now"),
        RoughTime::InTheFuture(units, 1) => format!("{} from now", units.singular()),  
        RoughTime::InTheFuture(units, count) => format!("{} {} from now", count, units.plural())  
    }
}

In [21]:
rough_time_to_english(RoughTime::InThePast(TimeUnit::Seconds, 1))

"second ago"

In [22]:
rough_time_to_english(RoughTime::InThePast(TimeUnit::Seconds, 42))

"42 seconds ago"

In [23]:
rough_time_to_english(RoughTime::JustNow)

"just now"

In [24]:
rough_time_to_english(RoughTime::InTheFuture(TimeUnit::Seconds, 42))

"42 seconds from now"

|      Pattern type       |        Example         |                                   Notes                                    |
| ----------------------- | ---------------------- | -------------------------------------------------------------------------- |
| Literal                 | `100`                  | Matches an exact value; the name of a const is also allowed                |
|                         | `"name"`               |                                                                            |
| Range                   | `0 ... 100`            | Matches any value in range, including the end value                        |
|                         | `'a' ... 'k'`          |                                                                            |
| Wildcard                | `_`                    | Matches any value and ignores it                                           |
| Variable                | `name`                 | Like _ but moves or copies the value into a new local variable             |
|                         | `mut count`            |                                                                            |
| ref variable            | `ref field`            | Borrows a reference to the matched value instead of moving or copying it   |
|                         | `ref mut field`        |                                                                            |
| Binding with subpattern | `val @ 0 ... 99`       | Matches the pattern to the right of @, using the variable name to the left |
|                         | `ref circle @`         |                                                                            |
|                         | `Shape::Circle { .. }` |                                                                            |
| Enum pattern	          | `Some(value)`          |                                                                            |
|                         | `None`                 |                                                                            |
|                         | `Message::Ok`          |                                                                            |
| Tuple pattern	          | `(key, value)`         |                                                                            |
|                         | `(r, g, b)`            |                                                                            |
| Struct pattern	      | `Color(r, g, b)`       |                                                                            |
|                         | `Point { x, y }`       |                                                                            |
|                         | `Card { suit: Clubs, rank: n }`  |                                                                  |
|                         | `Account { id, name, .. }`       |                                                                  | 
| Reference	              | `&value`               | Matches only reference values                                              |
|                         | `&(k, v)`	           |                                                                            |
| Multiple patterns	      | `'a' \| 'A'`           | In match only (not valid in let, etc.)                                     |
| Guard expression	      | `x if x * x <= r2`     | In match only (not valid in let, etc.)                                     |

### Literals, Variables and Wildcards in Patterns

In [25]:
let no_of_clicks = 4;

match no_of_clicks {
    0 => format!("No one clicked..."),
    1 => format!("Just one click..."),
    n => format!("{} clicks", n)
}

"4 clicks"

* Rust requires every match expression to handle all possible values, a wildcard is often required at the end

In [26]:
let http_code = HttpStatus::NotFound;

match http_code {
    HttpStatus::Ok => "Ok",
    _ => "Not Ok"
}

"Not Ok"

### Tuple Patterns

In [27]:
fn describe_point(x: i32, y:i32) -> &'static str {
    use std::cmp::Ordering::*;

    match (x.cmp(&0), y.cmp(&0)) {
        (Equal, Equal) => "at the origin",
        (_, Equal) => "on the x axis",
        (Equal, _) => "on the y axis",
        (Greater, Greater) => "in the first quadrant",
        (Less, Greater) => "in the second quadrant",
        _ => "somewhere else"
    }
}

In [28]:
describe_point(0, 42)

"on the y axis"

### Struct Patterns

In [73]:
#[derive(Debug, PartialEq)]
struct Point {
    x: i32, 
    y: i32
}

let location = Point{x:0, y:100};

match location {
    Point{x: 0, y: height} => format!("straight up {} meters", height),
    Point{x: x, y: y } => format!("at ({},{})", x, y)
}

"straight up 100 meters"

* Patterns like `Point { x: x, y: y }` are common when matching structs, and the redundant names are visual clutter, so Rust has a shorthand for this: `Point {x, y}` - this pattern still stores a point’s `x` field in a new local `x` and its `y` field in a new local `y`.

In [30]:
match location {
    Point{x: 0, y} => format!("straight up {} meters", y),
    Point{x, y} => format!("at ({},{})", x, y)
}

"straight up 100 meters"

* Matching larger structs

In [59]:
#[derive(Debug)]
struct Person {
    id: u64,
    first_name: String,
    last_name: String,
    age: u8,
    salary: f32,
    job: String
}

In [60]:
let person = Person{id: 665, first_name: "Jan".to_string(), last_name: "Kowalski".to_string(), age: 32, salary:9000.99, job: "Rust Developer".to_string() };

match person {
    Person{id: _, first_name, last_name, age: _, salary: _, job: _} => format!("{} {}", first_name, last_name)
}

"Jan Kowalski"

* We can skip uninteresting fields with `..`

In [61]:
let person = Person{id: 665, first_name: "Jan".to_string(), last_name: "Kowalski".to_string(), age: 32, salary:9000.99, job: "Rust Developer".to_string() };

match person {
    Person{first_name, last_name, ..} => format!("{} {}", first_name, last_name)
}

"Jan Kowalski"

### Reference Patterns

#### `ref` Patterns

* `ref` patterns borrow parts of a matched value

In [62]:
fn print_names(person: &Person) {
    print!("{} {}", person.first_name, person.last_name);    
}

In [63]:
let person = Person{ id: 665, first_name: "Jan".to_string(), last_name: "Kowalski".to_string(), age: 32, salary:9000.99, job: "Rust Developer".to_string() };

match person {
    Person{ job, .. } => { print_names(&person); println!("Works as {}", job) }
}

Error: use of deprecated method `core::str::<impl str>::trim_right_matches`: superseded by `trim_end_matches`

Error: borrow of partially moved value: `person`

In [64]:
let person = Person{ id: 665, first_name: "Jan".to_string(), last_name: "Kowalski".to_string(), age: 32, salary:9000.99, job: "Rust Developer".to_string() };

match person {
    Person{ ref job, .. } => { print_names(&person); println!(" Works as {}", job) }
}

Jan Kowalski Works as Rust Developer


()

In [65]:
fn to_upper(text: &mut String) {
    *text = text.to_uppercase();
}

let mut person = Person{ id: 665, first_name: "Jan".to_string(), last_name: "Kowalski".to_string(), age: 32, salary:9000.99, job: "Rust Developer".to_string() };

match person {
    Person{ref mut first_name, ref mut last_name, ..} => { to_upper(first_name); to_upper(last_name) }
}

person

Person { id: 665, first_name: "JAN", last_name: "KOWALSKI", age: 32, salary: 9000.99, job: "Rust Developer" }

#### & Pattern

* A pattern starting with `&` matches a reference.

In [101]:
#[derive(Debug, PartialEq)]
struct Circle { pos: Point, r: u32 }

impl Circle {
    fn center(&mut self) -> &mut Point {
        return &mut self.pos;
    }
}

let mut c = Circle{pos: Point{x: 100, y: 200}, r: 50};

match c.center() {
    &mut Point{x, y} => { format!("x={}, y={}", x, y) },
}

"x=100, y=200"

In [102]:
match c.center() {
    &mut Point{ref mut x, ..} => { *x = 10; },
}

c.center()

Point { x: 10, y: 200 }

In [128]:
#[derive(Debug)]
struct Item(String);

let items = [Item("one".to_string()), Item("two".to_string()), Item("three".to_string())];

{
    let mut iter = items.iter().peekable();

    match iter.peek() {
        Some(&item) => println!("item peeked: {:?}", item),
        None => println!("no item")
    }
}

item peeked: Item("one")


()

### Matching Multiple Possibilities

In [134]:
let text = String::from("\nBC");

{
    let mut chars = text.chars().peekable();

    let at_end = match chars.peek() {
        Some(&'\r') | Some(&'\n') | None => true,
        _ => false
    };

    at_end
}

true

In [ ]:
let text = String::from("\nBC");